<a href="https://colab.research.google.com/github/abalaji-blr/PyTorch/blob/main/FirstNN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [9]:
import torch
print (torch.__version__)

import torch.nn as nn
from torch.optim import SGD

from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs

1.9.0+cu102


# Define Neural Network

In [4]:
# define simple neural network 4 (inputs) - 8 (hidden) - 3 (output)
#
def build_model(inFeatures=4, hiddenDim=8, nbClasses=3):
  
	# construct a shallow, sequential neural network
	mlpModel = nn.Sequential(OrderedDict([
		("hidden_layer_1", nn.Linear(inFeatures, hiddenDim)),
		("activation_1", nn.ReLU()),
		("output_layer", nn.Linear(hiddenDim, nbClasses))
	]))

	# return the sequential model
	return mlpModel

In [5]:
def next_batch(inputs, targets, batchSize):
	# loop over the dataset
	for i in range(0, inputs.shape[0], batchSize):
		# yield a tuple of the current batched data and labels
		yield (inputs[i:i + batchSize], targets[i:i + batchSize])

In [6]:
# specify our batch size, number of epochs, and learning rate
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-2

# determine the device we will be using for training
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}...".format(DEVICE))

[INFO] training using cpu...


In [10]:
# generate a 3-class classification problem with 1000 data points,
# where each data point is a 4D feature vector
print("[INFO] preparing data...")
(X, y) = make_blobs(n_samples=1000, n_features=4, centers=3,
	cluster_std=2.5, random_state=95)

# create training and testing splits, and convert them to PyTorch
# tensors
(trainX, testX, trainY, testY) = train_test_split(X, y,
	test_size=0.15, random_state=95)
trainX = torch.from_numpy(trainX).float()
testX = torch.from_numpy(testX).float()
trainY = torch.from_numpy(trainY).float()
testY = torch.from_numpy(testY).float()

[INFO] preparing data...


In [11]:
# initialize our model and display its architecture
model = build_model().to(DEVICE)
print(model)

# initialize optimizer and loss function
opt = SGD(model.parameters(), lr=LR)
lossFunc = nn.CrossEntropyLoss()

Sequential(
  (hidden_layer_1): Linear(in_features=4, out_features=8, bias=True)
  (activation_1): ReLU()
  (output_layer): Linear(in_features=8, out_features=3, bias=True)
)


In [12]:
# loop through the epochs
for epoch in range(0, EPOCHS):
	# initialize tracker variables and set our model to trainable
	print("[INFO] epoch: {}...".format(epoch + 1))
	trainLoss = 0
	trainAcc = 0
	samples = 0
	model.train()

	# loop over the current batch of data
	for (batchX, batchY) in next_batch(trainX, trainY, BATCH_SIZE):
		# flash data to the current device, run it through our
		# model, and calculate loss
		(batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))
		predictions = model(batchX)
		loss = lossFunc(predictions, batchY.long())

		# zero the gradients accumulated from the previous steps,
		# perform backpropagation, and update model parameters
		opt.zero_grad()
		loss.backward()
		opt.step()

		# update training loss, accuracy, and the number of samples
		# visited
		trainLoss += loss.item() * batchY.size(0)
		trainAcc += (predictions.max(1)[1] == batchY).sum().item()
		samples += batchY.size(0)

	# display model progress on the current training batch
	trainTemplate = "epoch: {} train loss: {:.3f} train accuracy: {:.3f}"
	print(trainTemplate.format(epoch + 1, (trainLoss / samples),
		(trainAcc / samples)))

	# initialize tracker variables for testing, then set our model to
	# evaluation mode
	testLoss = 0
	testAcc = 0
	samples = 0
	model.eval()

	# initialize a no-gradient context
	with torch.no_grad():
		# loop over the current batch of test data
		for (batchX, batchY) in next_batch(testX, testY, BATCH_SIZE):
			# flash the data to the current device
			(batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))

			# run data through our model and calculate loss
			predictions = model(batchX)
			loss = lossFunc(predictions, batchY.long())

			# update test loss, accuracy, and the number of
			# samples visited
			testLoss += loss.item() * batchY.size(0)
			testAcc += (predictions.max(1)[1] == batchY).sum().item()
			samples += batchY.size(0)

		# display model progress on the current test batch
		testTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
		print(testTemplate.format(epoch + 1, (testLoss / samples),
			(testAcc / samples)))
		print("")

[INFO] epoch: 1...
epoch: 1 train loss: 0.713 train accuracy: 0.635
epoch: 1 test loss: 0.543 test accuracy: 0.720

[INFO] epoch: 2...
epoch: 2 train loss: 0.537 train accuracy: 0.692
epoch: 2 test loss: 0.424 test accuracy: 0.813

[INFO] epoch: 3...
epoch: 3 train loss: 0.419 train accuracy: 0.825
epoch: 3 test loss: 0.334 test accuracy: 0.913

[INFO] epoch: 4...
epoch: 4 train loss: 0.328 train accuracy: 0.925
epoch: 4 test loss: 0.264 test accuracy: 0.967

[INFO] epoch: 5...
epoch: 5 train loss: 0.259 train accuracy: 0.956
epoch: 5 test loss: 0.213 test accuracy: 0.980

[INFO] epoch: 6...
epoch: 6 train loss: 0.211 train accuracy: 0.965
epoch: 6 test loss: 0.177 test accuracy: 0.987

[INFO] epoch: 7...
epoch: 7 train loss: 0.177 train accuracy: 0.974
epoch: 7 test loss: 0.150 test accuracy: 0.987

[INFO] epoch: 8...
epoch: 8 train loss: 0.153 train accuracy: 0.978
epoch: 8 test loss: 0.131 test accuracy: 0.987

[INFO] epoch: 9...
epoch: 9 train loss: 0.135 train accuracy: 0.979
epoc